In [1]:
import analyze_rnn_predictions

/home/kaiolae/code/word_models_keras_test/WorldModels_hardmaru


In [2]:
#To access gym
import sys
sys.path.append('/home/kaiolae/.virtualenvs/worldmodels/lib/python3.5/site-packages')

In [3]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML


def plot_movie_mp4(image_array):
    dpi = 2.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    #fig = plt.figure(figsize=(1,1), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=len(image_array))
    display(HTML(anim.to_html5_video()))

In [4]:
from collections import namedtuple

#Loading RNNs to analyze
rnn1_analyzer = analyze_rnn_predictions.RNNAnalyzer("tf_rnn_1mixtures", 1, 0.1)
rnn4_analyzer = analyze_rnn_predictions.RNNAnalyzer("tf_rnn_4mixtures", 4, 0.1)
rnn8_analyzer = analyze_rnn_predictions.RNNAnalyzer("tf_rnn_8mixtures", 8, 0.1)
rnn16_analyzer = analyze_rnn_predictions.RNNAnalyzer("tf_rnn_16mixtures", 16, 0.1)

INFO:tensorflow:Model using cpu.
model using cpu
input dropout mode = False
output dropout mode = False
recurrent dropout mode = False
INFO:tensorflow:Model using cpu.
model using cpu
input dropout mode = False
output dropout mode = False
recurrent dropout mode = False
INFO:tensorflow:Model using cpu.
model using cpu
input dropout mode = False
output dropout mode = False
recurrent dropout mode = False
INFO:tensorflow:Model using cpu.
model using cpu
input dropout mode = False
output dropout mode = False
recurrent dropout mode = False


In [5]:
#Get a start video
#Getting data to feed into the VAE and RNN
import numpy as np
import os
# load preprocessed data
raw_data = np.load(os.path.join("series", "series.npz"))
raw_data_mu = raw_data["mu"]
raw_data_logvar = raw_data["logvar"]
raw_data_action =  raw_data["action"]

single_action_sequence = raw_data_action[16]
single_latent_sequence = raw_data_mu[16]
print("Actions length: ", len(single_action_sequence))
print("Latent vectors length: ", len(single_latent_sequence))
num_frames = len(single_latent_sequence)



#Decode all 100 vectors, store as video
#Visualize video.

Actions length:  177
Latent vectors length:  177


In [6]:
#Decode a sequence with the VAE and visualize it
def decode_and_visualize(latent_vector_sequence, rnn_analyzer):
    plot_movie_mp4(rnn_analyzer.decode_with_vae(latent_vector_sequence))

### Visualizing one sequence of latent vectors from the training data

In [ ]:
#From the actual sampled images. All loss here is due to VAE.
decode_and_visualize(single_latent_sequence, rnn1_analyzer)

In [ ]:
#Generating a dream.
action = 0 #Consider random action. Or, when predicting step by step, use the actual action
dreamed_zs1 = []
dreamed_z = rnn1_analyzer.predict_one_step(action, previous_z=single_latent_sequence[0])
dreamed_zs1.append(dreamed_z)
for i in range(100):
    dreamed_zs1.append(rnn1_analyzer.predict_one_step(action))
    

In [ ]:
#Visualizing the dream
decode_and_visualize(dreamed_zs1, rnn1_analyzer)

In [ ]:
#Generating a dream with 16 mixtures
action = 0 #Consider random action. Or, when predicting step by step, use the actual action
dreamed_zs16 = []
dreamed_z = rnn16_analyzer.predict_one_step(action, previous_z=single_latent_sequence[0])
dreamed_zs16.append(dreamed_z)
for i in range(100):
    dreamed_zs16.append(rnn16_analyzer.predict_one_step(action))
#Visualizing the dream
decode_and_visualize(dreamed_zs16, rnn16_analyzer)

# 

In [ ]:
#TODO Without reset, I got the expected result. But with reset, differences between RNNs disappear.
#Do differences for some reason only appear after a (large) number of iterations??
#One solution might be to run 1000 timesteps, to ensure we get into the internal state that
#exposes differences.
#Also, randomizing initial_z could help.
#Yes, longer sequences seemed to work.
from test_creativity_stability import count_events_on_trained_rnn
#Comparing performance of different models
initial_z = single_latent_sequence[0]
EPISODE_LENGTHS = 1000
zero_actions = [0 for i in range(EPISODE_LENGTHS)]

fb1=[]
m1=[]
fb8=[]
m8=[]
fb4=[]
m4=[]
fb16=[]
m16=[]

NUM_STATISTICAL_TESTS = 20

for i in range(NUM_STATISTICAL_TESTS):
    print(i)
    e1 = count_events_on_trained_rnn(rnn1_analyzer.vae, rnn1_analyzer, initial_z, zero_actions, EPISODE_LENGTHS)
    fb1.append(e1["num_fireballs"])
    m1.append(e1["num_monsters"])
    rnn1_analyzer._reset(initial_z)
    
    e8 = count_events_on_trained_rnn(rnn8_analyzer.vae, rnn8_analyzer, initial_z, zero_actions, EPISODE_LENGTHS)
    fb8.append(e8["num_fireballs"])
    m8.append(e8["num_monsters"])
    rnn8_analyzer._reset(initial_z)
    
    e4 = count_events_on_trained_rnn(rnn4_analyzer.vae, rnn4_analyzer, initial_z, zero_actions, EPISODE_LENGTHS)
    fb4.append(e4["num_fireballs"])
    m4.append(e4["num_monsters"])
    rnn4_analyzer._reset(initial_z)
    
    e16 = count_events_on_trained_rnn(rnn16_analyzer.vae, rnn16_analyzer, initial_z, zero_actions, EPISODE_LENGTHS)
    fb16.append(e16["num_fireballs"])
    m16.append(e16["num_monsters"])
    rnn16_analyzer._reset(initial_z)
    


In [ ]:
print("fb1:", sum(fb1)/float(len(fb1)))
print("fb4:", sum(fb4)/float(len(fb4)))
print("fb8:", sum(fb8)/float(len(fb8)))
print("fb16:", sum(fb16)/float(len(fb16)))

print("m1:", sum(m1)/float(len(m1)))
print("m4:", sum(m4)/float(len(m4)))
print("m8:", sum(m8)/float(len(m8)))
print("m16:", sum(m16)/float(len(m16)))
#TODO Store and show some random videos, to help interpret these results.
#TODO box plots?
#Those fireball numbers suggest increased creativity with higher numbers of mixtures.
#Now, how to measure stability? Maybe predicting N (=1 initially) frames ahead, and measuring diff btw z-vectors,
#OR diff btw fireball/monster metrics?

In [ ]:
print(fb1)

In [12]:
#Testing differences between real and predicted images. Rather than dreaming up many steps ahead, we here
#Dream 1 step ahead, and compare to what actually happened. Tests stability of the prediction.
mm1 = []
mf1= []
im1 = []
if1 = []

mm4 = []
mf4= []
im4 = []
if4 = []

mm8 = []
mf8= []
im8 = []
if8 = []

mm16 = []
mf16= []
im16 = []
if16 = []
initial_z = single_latent_sequence[0]

NUM_STATISTICAL_TESTS = 20
from test_creativity_stability import count_differences_between_reality_and_prediction

for i in range(NUM_STATISTICAL_TESTS):
    print(i)
    e1 = count_differences_between_reality_and_prediction(rnn1_analyzer.vae, rnn1_analyzer, single_latent_sequence, single_action_sequence)
    mf1.append(e1["missing_fireballs"])
    if1.append(e1["imagined_fireballs"])
    rnn1_analyzer._reset(initial_z)
    
    e8 = count_differences_between_reality_and_prediction(rnn8_analyzer.vae, rnn8_analyzer, single_latent_sequence, single_action_sequence)
    mf8.append(e8["missing_fireballs"])
    if8.append(e8["imagined_fireballs"])
    rnn8_analyzer._reset(initial_z)
    
    e4 = count_differences_between_reality_and_prediction(rnn4_analyzer.vae, rnn4_analyzer, single_latent_sequence, single_action_sequence)
    mf4.append(e4["missing_fireballs"])
    if4.append(e4["imagined_fireballs"])
    rnn4_analyzer._reset(initial_z)
    
    e16 = count_differences_between_reality_and_prediction(rnn16_analyzer.vae, rnn16_analyzer, single_latent_sequence, single_action_sequence)
    mf16.append(e16["missing_fireballs"])
    if16.append(e16["imagined_fireballs"])
    rnn16_analyzer._reset(initial_z)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [13]:
print("mf1:", sum(mf1)/float(len(mf1)))
print("mf4:", sum(mf4)/float(len(mf4)))
print("mf8:", sum(mf8)/float(len(mf8)))
print("mf16:", sum(mf16)/float(len(mf16)))

print("if1:", sum(if1)/float(len(if1)))
print("if4:", sum(if4)/float(len(if4)))
print("if8:", sum(if8)/float(len(if8)))
print("if16:", sum(if16)/float(len(if16)))

#Preliminary conclusion: Many mixtures (8/16) give more creativity, being able to "imagine" more fireballs,
#but also more inaccuracies, by imagining fireballs that shouldnt be there.

mf1: 3.55
mf4: 10.15
mf8: 3.85
mf16: 3.85
if1: 1.65
if4: 1.55
if8: 3.4
if16: 3.3
